In [2]:
import sys
import os
from pathlib import Path

# Caminho absoluto para a raiz do projeto (um nível acima de notebooks)
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)


In [3]:
from config.utils import *


In [40]:
path = os.path.join(PATHS['BRONZE_LAYER'],"ars_brl_20250701.json")

In [41]:
pd.read_json(path)

,data,valor
0,01/07/2015,0.02534
1,02/07/2015,0.02531
2,03/07/2015,0.02549
3,06/07/2015,0.02564
4,07/07/2015,0.02603
...,...,...
2501,25/06/2025,0.03806
2502,26/06/2025,0.03820
2503,27/06/2025,0.03782
2504,30/06/2025,0.03785


In [16]:
INDICADORES.keys()

dict_keys(['IPCA', 'SELIC_META', 'IBC_BR', 'DESEMPREGO', 'INADIMPLENCIA_PF', 'ARS_BRL', 'USD_BRL', 'EUR_BRL', 'GBP_BRL', 'JPY_BRL', 'CHF_BRL', 'CNY_BRL'])

In [105]:
PATHS['BRONZE_LAYER']

'C:\\Users\\BROliveiWa12\\OneDrive - NESTLE\\Documents\\Estudos\\Engenharia de Dados\\economic-indicators-pipeline\\data\\bronze'

In [148]:
def ler_dados_cotacao(moeda: str, path: str = PATHS['BRONZE_LAYER'], data: str = DATA_FINAL):
    """
    Lê um arquivo JSON de cotação e retorna um DataFrame.

    Args:
        path (str): Caminho da pasta onde está o JSON.
        moeda (str): Nome da moeda (ex: 'USD_BRL').
        data (datetime, opcional): Data desejada. Default: DATA_FINAL.

    Returns:
        [pd.DataFrame]: DataFrame formatado ou None se houver erro.
    """

    try:
        moeda = moeda.lower()
        data_str = pd.to_datetime(data, format="%d/%m/%Y").strftime("%Y%m%d")
        nome_arquivo = f"{moeda}_{data_str}.json"
        path_arquivo = os.path.join(path, nome_arquivo)

        logger.info(f"Lendo arquivo: {path_arquivo}")

        df = pd.read_json(path_arquivo)

       
        df.rename(columns={'valor': moeda}, inplace=True)
        df = df.astype({'data': 'datetime64', moeda.lower(): 'float64'}, errors='ignore')

      
        return df

    except FileNotFoundError:
        logger.warning(f"Arquivo não encontrado: {nome_arquivo}")
    except Exception as e:
        logger.error(f"Erro ao ler {moeda} - {data_str}: {e}", exc_info=True)

    return None

In [149]:
moedas = ['ARS_BRL', 'USD_BRL', 'EUR_BRL', 'GBP_BRL', 'JPY_BRL', 'CHF_BRL', 'CNY_BRL']

df_cotacoes = ler_dados_cotacao(moeda='ars_brl',path=PATHS['BRONZE_LAYER'])

2025-07-13 01:37:18.458 | INFO     | __main__:ler_dados_cotacao:20 - Lendo arquivo: C:\Users\BROliveiWa12\OneDrive - NESTLE\Documents\Estudos\Engenharia de Dados\economic-indicators-pipeline\data\bronze\ars_brl_20250712.json


In [150]:
df_cotacoes

,data,ars_brl
0,13/07/2015,0.02560
1,14/07/2015,0.02530
2,15/07/2015,0.02544
3,16/07/2015,0.02529
4,17/07/2015,0.02566
...,...,...
2501,07/07/2025,0.03739
2502,08/07/2025,0.03716
2503,09/07/2025,0.03727
2504,10/07/2025,0.03784


In [151]:
def salvar_dados_cotacao(dados: pd.DataFrame, moeda: str):
    """
    Salva os dados brutos da API na camada silver.

    Args:
        dados (DataFrame): Dados retornados da API.
        nome_cotacao (str): Nome da moeda de cotação.

    Returns:
        None
    """
    try:
        nome_arquivo = f"{moeda.lower()}.parquet"
        caminho_arquivo = os.path.join(PATHS["SILVER_LAYER"],nome_arquivo)
        print(caminho_arquivo)
        dados.to_parquet(caminho_arquivo)

        
        logger.info(f"Indicacdor {moeda} salvo no arquivo: {nome_arquivo}")

    except Exception as e:
        logger.error(f"Erro ao salvar os dados na camada silver: {str(e)}")
        raise e

In [152]:
salvar_dados_cotacao(df_cotacoes,'ARS_BRL')

2025-07-13 01:37:24.500 | INFO     | __main__:salvar_dados_cotacao:19 - Indicacdor ARS_BRL salvo no arquivo: ars_brl.parquet


C:\Users\BROliveiWa12\OneDrive - NESTLE\Documents\Estudos\Engenharia de Dados\economic-indicators-pipeline\data\silver\ars_brl.parquet


In [154]:
for nome_moeda, parametros in list(INDICADORES.items())[-7:]:
   dados_cotacao = ler_dados_cotacao(moeda=nome_moeda)
   salvar_dados_cotacao(dados=dados_cotacao,moeda=nome_moeda)


2025-07-13 01:37:44.997 | INFO     | __main__:ler_dados_cotacao:20 - Lendo arquivo: C:\Users\BROliveiWa12\OneDrive - NESTLE\Documents\Estudos\Engenharia de Dados\economic-indicators-pipeline\data\bronze\ars_brl_20250712.json
2025-07-13 01:37:45.012 | INFO     | __main__:salvar_dados_cotacao:19 - Indicacdor ARS_BRL salvo no arquivo: ars_brl.parquet
2025-07-13 01:37:45.014 | INFO     | __main__:ler_dados_cotacao:20 - Lendo arquivo: C:\Users\BROliveiWa12\OneDrive - NESTLE\Documents\Estudos\Engenharia de Dados\economic-indicators-pipeline\data\bronze\usd_brl_20250712.json
2025-07-13 01:37:45.049 | INFO     | __main__:salvar_dados_cotacao:19 - Indicacdor USD_BRL salvo no arquivo: usd_brl.parquet
2025-07-13 01:37:45.050 | INFO     | __main__:ler_dados_cotacao:20 - Lendo arquivo: C:\Users\BROliveiWa12\OneDrive - NESTLE\Documents\Estudos\Engenharia de Dados\economic-indicators-pipeline\data\bronze\eur_brl_20250712.json
2025-07-13 01:37:45.087 | INFO     | __main__:salvar_dados_cotacao:19 - Ind

C:\Users\BROliveiWa12\OneDrive - NESTLE\Documents\Estudos\Engenharia de Dados\economic-indicators-pipeline\data\silver\ars_brl.parquet
C:\Users\BROliveiWa12\OneDrive - NESTLE\Documents\Estudos\Engenharia de Dados\economic-indicators-pipeline\data\silver\usd_brl.parquet
C:\Users\BROliveiWa12\OneDrive - NESTLE\Documents\Estudos\Engenharia de Dados\economic-indicators-pipeline\data\silver\eur_brl.parquet
C:\Users\BROliveiWa12\OneDrive - NESTLE\Documents\Estudos\Engenharia de Dados\economic-indicators-pipeline\data\silver\gbp_brl.parquet
C:\Users\BROliveiWa12\OneDrive - NESTLE\Documents\Estudos\Engenharia de Dados\economic-indicators-pipeline\data\silver\jpy_brl.parquet
C:\Users\BROliveiWa12\OneDrive - NESTLE\Documents\Estudos\Engenharia de Dados\economic-indicators-pipeline\data\silver\chf_brl.parquet


2025-07-13 01:37:45.203 | INFO     | __main__:salvar_dados_cotacao:19 - Indicacdor CHF_BRL salvo no arquivo: chf_brl.parquet
2025-07-13 01:37:45.204 | INFO     | __main__:ler_dados_cotacao:20 - Lendo arquivo: C:\Users\BROliveiWa12\OneDrive - NESTLE\Documents\Estudos\Engenharia de Dados\economic-indicators-pipeline\data\bronze\cny_brl_20250712.json
2025-07-13 01:37:45.243 | INFO     | __main__:salvar_dados_cotacao:19 - Indicacdor CNY_BRL salvo no arquivo: cny_brl.parquet


C:\Users\BROliveiWa12\OneDrive - NESTLE\Documents\Estudos\Engenharia de Dados\economic-indicators-pipeline\data\silver\cny_brl.parquet


In [ ]:
teste = obter_dados_api(INDICADORES['IPCA'])

In [ ]:
#INDICADORES['IPCA']['URL'],
INDICADORES['DESEMPREGO']['URL']

In [ ]:
teste[0]['resultados'][0]['series'][0]['serie']

In [ ]:
salvar_dados_api(teste, 'ipca')

In [ ]:
for nome_indicador, config in INDICADORES.values():
    try:
        dados = obter_dados_api(indicador)
        salvar_dados_api(dados, indicador['NOME'])
    except Exception as e:
        logger.error(f"Erro ao processar o indicador {indicador['NOME']}: {str(e)}")

In [ ]:

Cotaçao cambio
response = requests.get("https://api.bcb.gov.br/dados/serie/bcdata.sgs.21619/dados?formato=json&dataInicial=29/06/2015")
response.json()

In [ ]:
url = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.432/dados?formato=json&dataInicial=01/07/2015&dataFinal=01/07/2025"

response = requests.get(url)
response.raise_for_status()  
dados_cambio = response.json()

In [ ]:
dados_cambio